# Capture stream from fluentbit

In [1]:
# Import modules
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split


In [2]:
spark = SparkSession.builder.appName("StreamingDemo").getOrCreate()

23/03/28 20:38:47 WARN Utils: Your hostname, codespaces-16de0b resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/03/28 20:38:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/28 20:38:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
lines = spark.readStream.format("socket").option("host", "local").option("port", 9999).load()

23/03/28 20:38:55 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
words = lines.select(explode(split(lines.value, " ")).alias("word"))


In [ ]:
# Create a DStream that will connect to hostname:port, like localhost:9999
ssc.start() 
ssc.awaitTermination() 
lines = ssc.socketTextStream("localhost", 9999)
logging.info(lines)


# preprocess a .gz file to isolate one pods worth of data

In [ ]:
def gz2df(path):
    with gzip.open(path, 'rb') as f:
        """
        Convert .gz log files into dataframe
        """
        content = f.read().decode('utf-8').split('\n')
        list_rows = []

        for i in range(len(content)-1):
            row = content[i].split(' ')
            list_rows.append(row)
        f.close()

    df = pd.DataFrame(list_rows, columns=['log_timestamp', 'data'])

    return df

def get_type(x):
    idx1 = x.find('"Type":') + 8
    idx2 = x.find(',', idx1) - 1
    rec_type = x[ idx1:idx2 ]

    return rec_type

def filter_by_rec_type(df, rec_type): 
    return df[df.rec_type == rec_type]

def explode_df(df):
    return pd.concat([df.log_timestamp, pd.json_normalize(df.data)], axis=1)


In [ ]:
streamed_data_file = "./node1.gz"

df = gz2df(streamed_data_file)
df['rec_type'] = df.data.apply(get_type)
df['data'] = df.data.apply(json.loads)



list_of_df_rectypes = {}  

for rec_type in df.rec_type.unique():
    print(rec_type)
    rec_type_df = filter_by_rec_type(df, rec_type)
    df_exploded = explode_df(rec_type_df)
    list_of_df_rectypes[rec_type]=df_exploded
    


In [ ]:
df_pod = list_of_df_rectypes["Pod"]

In [ ]:
if("open5gs-upf" in df_pod.PodName.unique()):
    print("upf data exists")


upf_df = df_pod[df_pod["PodName"]=="open5gs-upf" ]

In [ ]:
## cleaning up the nans in the dataframe

print(upf_df['pod_cpu_usage_total'].isna().sum())
print(len(upf_df))

upf_df = upf_df.dropna(subset=['pod_cpu_usage_total', 'pod_memory_max_usage' ])

# Model Time

In [ ]:
##setting the limits and requests
limit_cpu = max( upf_df[["pod_cpu_usage_total"]].values.tolist() )
request_cpu = np.mean( upf_df[["pod_cpu_usage_total"]].values.tolist() ) 

limit_memory =   max( upf_df[["pod_memory_max_usage"]].values.tolist() )
request_memory = np.mean( upf_df[["pod_memory_max_usage"]].values.tolist() ) 

# Time to postprocess

In [ ]:
logging.info(f'limit_cpu: {limit_cpu}')
logging.info(f'request_cpu: {request_cpu}')
logging.info(f'limit_memory: {limit_memory}')
logging.info(f'request_memory: {request_memory}')
ssc.stop()